# **Decision Tree Algorithm using Information Gain**

## Importing necessary libraries

In [10]:
import pandas as pd
import numpy as np
import random

## Defining functions

### Sub-functions

In [11]:
# algorithms


def calculateEntropy(dataframe):

  labels, probabilities = np.unique(dataframe.loc[:,"Label"], return_counts=True)
  probabilities = probabilities/sum(probabilities)
  return -np.dot(probabilities,np.log2(probabilities))

def calculateGain(dataframe,feature):

  parentEntropy = calculateEntropy(dataframe)
  yesDataframe = dataframe[dataframe[feature]=='y']
  noDataframe = dataframe[dataframe[feature]=='n']
  numOfParent = len(dataframe)

  averageYesEntropy =  calculateEntropy(yesDataframe)*(len(yesDataframe)/numOfParent)
  averageNoEntropy =  calculateEntropy(noDataframe)*(len(noDataframe)/numOfParent)

  return parentEntropy-(averageYesEntropy+averageNoEntropy)


def chooseFeature(dataframe,cols):
  entropy = calculateEntropy(dataframe)
  bestFeature=cols[0]
  bestGain = calculateGain(dataframe,cols[0])
  for i in cols:
    tempDataframe = dataframe.copy()
    if(bestGain<calculateGain(tempDataframe,i)):
    
      bestFeature=i
    
  return bestFeature

def checkPure(space):
  for d in space:
    if(d[0].loc[:,'Label'].nunique()!=1):
      return False
  return True
    

### Building the tree, the setting the classifier and implementing the classification function

In [12]:
def Decision_Tree_Classifier(dataframe):
  rules = []
  level=0
  space =[[dataframe,""]]

  columns = dataframe.columns.copy().tolist()[1:]
  
  maxDecision = 500
  while(((not checkPure(space))and (level<maxDecision) )):
    #print(columns)
    
    level+=1
    record = space.pop(0)
    dataframe = record[0] 
    if(len(dataframe)==0):
      continue

      # perfectly classified 
    if(dataframe.loc[:,'Label'].nunique()==1):
      rules.append([record[1],dataframe['Label'].unique()[0]])
      #print(dataframe.head())
      continue

    bestFeature = chooseFeature(dataframe,columns)
    #columns.remove(bestFeature)
    path = record[1]+ bestFeature 
      
    space.append([dataframe[dataframe[bestFeature]=='y'],path+":y "])
    space.append([dataframe[dataframe[bestFeature]=='n'],path+":n "])

  # get pure rules

  return rules


In [13]:

def setClassifier(rules):
  classifier = []
  for r in rules:
    c = []
    path = r[0].split(" ")
    path=path[:len(path)-1]
    for f in path:
      c.append(f.split(":"))
      
    classifier.append([c,r[1]])

  return classifier 

def classify(classifier,record):
  for i in classifier:
    rule = i[0]
    prediction = i[1]
    state = True
    for j in rule:
      if(record[j[0]]!=j[1]):
        state=False
        break 
    if(state==True):
      return prediction

  return "Can not classify"
          

def getError(classifier,testData):
  i=0
  for index, row in testData.iterrows():
    if(classify(classifier,row )!=row['Label']):
      i+=1
  return i/len(testData)




## Reading and Preprocessing The Data

### Reading the data from the file

In [14]:
# reading data

data = []

file = open("house-votes-84.txt","r")

for  line in file:

  record = line[:len(line)-1] .split(",")
  data.append(record)

file.close()

### Creating a Dataframe

In [15]:

dataframe = pd.DataFrame(data, columns=["Label","Issue_1","Issue_2","Issue_3","Issue_4","Issue_5","Issue_6","Issue_7","Issue_8","Issue_9","Issue_10","Issue_11","Issue_12","Issue_13","Issue_14","Issue_15","Issue_16"])


### Preprocessing the data to remove none values

In [16]:
# preprocess the data
for i in range (1,17):
  col = "Issue_"+str(i)
  numNo = len(dataframe[dataframe[col]=='n'])
  numYes = len(dataframe[dataframe[col]=='y'])
  if(numNo>numYes):
    dataframe.loc[dataframe[col] == '?',col] = 'n'

  else:
    dataframe.loc[dataframe[col] == '?',col] = 'y'


## Defining the run functions

In [17]:

def run(train,test):
  decisionTree=Decision_Tree_Classifier(train) 
  classifier = setClassifier(decisionTree)
  return len(classifier),getError(classifier,test)


def run_randomly(dataframe,partition):
  acc = [1000,0]
  nr  = [1000,0]

  for i in range(5):
    seed = random.randint(0, 30)
    train=dataframe.sample(n=int(partition*len(dataframe)), replace=False,random_state=seed) 
    test=dataframe.drop(train.index) 
    numOfRules,error = run(train,test)
    if(acc[0]>error):
      acc[0]=error
    if(acc[1]<error):
      acc[1]=error

    if(nr[0]>numOfRules):
      nr[0]=numOfRules
    if(nr[1]<numOfRules):
      nr[1]=numOfRules


    print("For sample "+str(i+1)+":  num of rules= "+ str(numOfRules) +", with error= "+str(error) +" with seed:"+str(seed))
  print("Min len of rules: "+str(nr[0])+", mean len of rules:"+str((nr[0]+nr[1])/2.0)+", max len of rules:"+str(nr[1]))
  print("Min error: "+str(acc[0])+", mean error:"+str((acc[0]+acc[1])/2.0)+", max error:"+str(acc[1]))  

  

## Running the Model over Different training and testing data

### Using 25% for training and 75% for testing not by setting the seed randomly

In [18]:

for i in range(5):
  train=dataframe.sample(frac =.25) 
  test=dataframe.drop(train.index) 
  numOfRules,error = run(train,test)
  print("For sample "+str(i+1)+":  num of rules= "+ str(numOfRules) +", with error= "+str(error))


For sample 1:  num of rules= 18, with error= 0.19631901840490798
For sample 2:  num of rules= 29, with error= 0.1196319018404908
For sample 3:  num of rules= 15, with error= 0.17177914110429449
For sample 4:  num of rules= 13, with error= 0.21779141104294478
For sample 5:  num of rules= 28, with error= 0.2361963190184049


### Using 30% for training and 70% for testing

In [19]:
run_randomly(dataframe,0.3)


For sample 1:  num of rules= 18, with error= 0.16721311475409836 with seed:4
For sample 2:  num of rules= 22, with error= 0.2 with seed:26
For sample 3:  num of rules= 34, with error= 0.18688524590163935 with seed:17
For sample 4:  num of rules= 27, with error= 0.18688524590163935 with seed:10
For sample 5:  num of rules= 27, with error= 0.1377049180327869 with seed:23
Min len of rules: 18, mean len of rules:26.0, max len of rules:34
Min error: 0.1377049180327869, mean error:0.16885245901639345, max error:0.2


### Using 40% for training and 60% for testing

In [20]:
run_randomly(dataframe,0.4)


For sample 1:  num of rules= 28, with error= 0.26436781609195403 with seed:14
For sample 2:  num of rules= 39, with error= 0.13793103448275862 with seed:1
For sample 3:  num of rules= 39, with error= 0.19540229885057472 with seed:22
For sample 4:  num of rules= 35, with error= 0.11877394636015326 with seed:7
For sample 5:  num of rules= 29, with error= 0.2567049808429119 with seed:2
Min len of rules: 28, mean len of rules:33.5, max len of rules:39
Min error: 0.11877394636015326, mean error:0.19157088122605365, max error:0.26436781609195403


### Using 50% for training and 50% for testing

In [21]:
run_randomly(dataframe,0.5)


For sample 1:  num of rules= 37, with error= 0.15137614678899083 with seed:30
For sample 2:  num of rules= 46, with error= 0.1651376146788991 with seed:6
For sample 3:  num of rules= 50, with error= 0.13302752293577982 with seed:20
For sample 4:  num of rules= 52, with error= 0.10091743119266056 with seed:29
For sample 5:  num of rules= 50, with error= 0.0871559633027523 with seed:13
Min len of rules: 37, mean len of rules:44.5, max len of rules:52
Min error: 0.0871559633027523, mean error:0.1261467889908257, max error:0.1651376146788991


### Using 60% for training and 40% for testing

In [22]:
run_randomly(dataframe,0.6)


For sample 1:  num of rules= 40, with error= 0.15517241379310345 with seed:9
For sample 2:  num of rules= 48, with error= 0.13218390804597702 with seed:20
For sample 3:  num of rules= 48, with error= 0.13218390804597702 with seed:30
For sample 4:  num of rules= 58, with error= 0.14367816091954022 with seed:12
For sample 5:  num of rules= 49, with error= 0.1206896551724138 with seed:7
Min len of rules: 40, mean len of rules:49.0, max len of rules:58
Min error: 0.1206896551724138, mean error:0.13793103448275862, max error:0.15517241379310345


### Using 70% for training and 30% for testing

In [23]:
run_randomly(dataframe,0.7)

For sample 1:  num of rules= 59, with error= 0.13740458015267176 with seed:22
For sample 2:  num of rules= 46, with error= 0.1450381679389313 with seed:11
For sample 3:  num of rules= 79, with error= 0.061068702290076333 with seed:27
For sample 4:  num of rules= 61, with error= 0.15267175572519084 with seed:21
For sample 5:  num of rules= 54, with error= 0.13740458015267176 with seed:26
Min len of rules: 46, mean len of rules:62.5, max len of rules:79
Min error: 0.061068702290076333, mean error:0.10687022900763359, max error:0.15267175572519084


The error was the least when the train test split was 70%-30%